In [1]:
print("chroma db")

chroma db


In [2]:
# !pip install chromadb-client==1.0.0
# !pip install --upgrade chromadb==1.0.0
#!pip install --upgrade langchain-chroma

In [3]:
# !pip install chromadb tiktoken transformers setence_transformers pypdf

In [4]:
# !pip install faiss-cpu

In [5]:
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_community.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain_community.vectorstores import Chroma # type: ignore
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_core.documents import Document # type: ignore
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import tiktoken
import warnings
warnings.filterwarnings("ignore")

C:\Users\euiyoung.hwang\Git_Workspace\Vector_DB_with_LLM\.venv\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [6]:
os.environ['CURL_CA_BUNDLE'] = ''

In [7]:
docs = [
            Document(page_content="사과", metadata=dict(page=1)),
            Document(page_content="애플", metadata=dict(page=1)),
            Document(page_content="바나나", metadata=dict(page=2)),
            Document(page_content="오렌지", metadata=dict(page=2)),
            Document(page_content="고양이", metadata=dict(page=3)),
            Document(page_content="야옹", metadata=dict(page=3)),
            Document(page_content="강아지", metadata=dict(page=4)),
            Document(page_content="멍멍", metadata=dict(page=4)),
            Document(page_content="해", metadata=dict(page=5)),
            Document(page_content="달", metadata=dict(page=5)),
            Document(page_content="물", metadata=dict(page=6)),
            Document(page_content="불", metadata=dict(page=6)),
            Document(page_content="apple", metadata=dict(page=7)),
]

In [8]:
embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
    )

In [9]:
tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [10]:
print("In Progressing..")

In Progressing..


In [11]:
# -- Reference: https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('./gsat_170823.pdf')
pages = loader.load_and_split()

## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0,length_function=tiktoken_len)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 75 65536 (offset 0)
Ignoring wrong pointing object 85 65536 (offset 0)
Ignoring wrong pointing object 94 65536 (offset 0)
Ignoring wrong pointing object 98 65536 (offset 0)
Ignoring wrong pointing object 102 65536 (offset 0)
Ignoring wrong pointing object 109 65536 (offset 0)
Ignoring wrong pointing object 119 65536 (offset 0)
Ignoring wrong pointing object 132 65536 (offset 0)
Ignoring wrong pointing object 148 65536 (offset 0)
Ignoring wrong pointing object 176 65536 (offset 0)
Ignoring wrong pointing object 188 65536 (offset 0)
Ignoring wrong pointing object 192 65536 (offset 0)
Ignoring wrong pointing object 199 65536 (offset 0)
Ignoring wrong pointing object 203 65536 (offset 0)
Ignoring wrong pointing object 207 65536 (offset 0)
Ignoring wrong pointing object 211 65536 (offset 0)
Ignoring wrong pointing object 215 65536 (offset 0)
Ignoring wrong pointing object 219 65536 (offset 0)
Ignoring wrong poi

In [12]:
# print(docs)

In [13]:
# db = Chroma.from_documents(docs, embeddings)
db = FAISS.from_documents(docs, embeddings)

In [14]:
question = '삼성전자의 주요 사업영역은?'
r_docs = db.similarity_search(question)
r_docs

[Document(metadata={'source': './gsat_170823.pdf', 'page': 9}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문\n① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공\n② 네트워크 : 뛰어난 기술력과 성공적인 경험을 바탕으로 차세대 네트워크 인프라와 솔루션을 \n공급\n⑶ Device Solutions (DS) 부문\n① 메모리 : 세계 최고 성능의 최첨단 그린메모리로 더욱 편리한 세상을 향해 나아감'),
 Document(metadata={'source': './gsat_170823.pdf', 'page': 0}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분

In [15]:
print("Done..")

Done..
